In [ ]:
import json
from tqdm import tqdm
import time
import os
import sys
import matplotlib.pyplot as plt

import cv2
import numpy as np
import pandas as pd

import laspy

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from plotly.offline import init_notebook_mode
from plotly.offline import iplot
from plotly.offline import plot
from plotly import graph_objs as go
from plotly.io import write_html
init_notebook_mode()


def make_graph_2d(x, y, color):
    graph = go.Scatter({
        "x": x,
        "y": y,
        "mode": "markers",
        "marker": {
            "size": 1,
            "color": color,
        },
    })
    return graph

def make_graph_3d(x, y, z, color):
    graph = go.Scatter3d({
        "x": x,
        "y": y,
        "z": z,
        "mode": "markers",
        "marker": {
            "size": 1,
            "color": color,
        },
    })
    return graph

def make_vector_3d(xyz, color, name=''):
    x,y,z = xyz
    graph = go.Scatter3d({
        "x": [0,x],
        "y": [0,y],
        "z": [0,z],
        "mode": "lines",
        "line": {
            "color": color,
        },
        'name':name
    })
    return graph

def _make_fig(graphs, height=None):
    scene_axis = {
        "showticklabels": False,
        "showgrid": False,
        "zeroline": False,
        "showline": True,
        "linewidth": 1.5,
        "linecolor":"#555",
        "mirror": True,
        "title_text": "",
    }
    
    params = {
        "scene": {
            "domain": {"x": [0.0, 1.0], "y": [0.25, 1.0]},
            "aspectmode": "data",
            "xaxis": scene_axis,
            "yaxis": scene_axis,
            "zaxis": scene_axis,
        },
        "paper_bgcolor": "#222",
        "plot_bgcolor": "#222",
    }
    if height is not None:
        params['height'] = height

    fig_layout = go.Layout(params)


    fig = go.Figure({
        "data": graphs,
        "layout": fig_layout,
    })
    return fig

def show_graphs(graphs, height=800):
    fig = _make_fig(graphs, height=height)

    iplot(fig)

def write_html_graphs(graphs, name):
    fig = _make_fig(graphs)

    write_html(fig, name)

# Load dataset

We need to find 1x1km of switzerland chunks with rich landscape.

[Lidar data download](https://www.swisstopo.admin.ch/en/geodata/height/surface3d.html#download)

[Aero photos download]()

In [ ]:
base_dir = '/home/safic2/A_WORK_PROJECTS/F_MY_PROJECTS/DIGITAL_BREAKTHROUGH_2021/mks_final_minenergo/RES/swiss_lidar_and_surface'

os.path.exists(base_dir)

# Load lidar data

In [ ]:
ds = pd.read_csv(f'{base_dir}/lidar_data.csv', names=['url'])
ds

In [ ]:
for _, row in tqdm(ds.iterrows()):
    url = row['url']
    os.system(f'wget {url}')

In [ ]:
lidar_zip_dir = f'{base_dir}/src/lidar_zip'

if not os.path.exists(lidar_zip_dir):
    os.makedirs(lidar_zip_dir)

In [ ]:
for file in os.listdir('.'):
    if file.startswith('swisssurface3d'):
        os.system(f'mv {file} {lidar_zip_dir}')

In [ ]:
for file in os.listdir(lidar_zip_dir):
    os.system(f'unzip {lidar_zip_dir}/{file}')

In [ ]:
lidar_dir = f'{base_dir}/src/lidar'

if not os.path.exists(lidar_dir):
    os.makedirs(lidar_dir)

In [ ]:
for file in os.listdir('.'):
    if file.endswith('.las'):
        os.system(f'mv {file} {lidar_dir}')

# Load image data

In [ ]:
ds = pd.read_csv(f'{base_dir}/image_data.csv', names=['url'])
ds

In [ ]:
for _, row in tqdm(ds.iterrows()):
    url = row['url']
    os.system(f'wget {url}')

In [ ]:
image_dir = f'{base_dir}/src/image'

if not os.path.exists(image_dir):
    os.makedirs(image_dir)

In [ ]:
for file in os.listdir('.'):
    if file.startswith('swissimage'):
        os.system(f'mv {file} {image_dir}')

## List lidar files

In [ ]:
lidar_dir = f'{base_dir}/src/lidar'

os.path.exists(base_dir)

In [ ]:
lidar_file_names = os.listdir(lidar_dir)

print(lidar_file_names)

## List image files

In [ ]:
image_dir = f'{base_dir}/src/image'

os.path.exists(base_dir)

In [ ]:
image_file_names = os.listdir(image_dir)

print(image_file_names)

# Debug lidar algorithms

## Load data

In [ ]:
# file = '/home/safic2/A_WORK_PROJECTS/F_MY_PROJECTS/DIGITAL_BREAKTHROUGH_2021/mks_final_minenergo/RES/swiss_lidar_and_surface/old_src/lidar/2544_1183.las'
file = f'{lidar_dir}/{lidar_file_names[-1]}'

with laspy.open(file) as inf:
    for points in inf.chunk_iterator(10**10):        
        x = points.x.copy()
        y = points.y.copy()
        z = points.z.copy()
        c = points.classification.copy()

In [ ]:
arr = np.stack([x,y,z,c], axis=1)

ds = pd.DataFrame(arr, columns=['x','y','z','class'])
ds

In [ ]:
ds['x'] -= np.min(ds['x'])
ds['y'] -= np.min(ds['y'])
ds['z'] -= np.min(ds['z'])

print(np.min(ds['x']), np.max(ds['x']))
print(np.min(ds['y']), np.max(ds['y']))
print(np.min(ds['z']), np.max(ds['z']))

In [ ]:
def get_crop(ix, iy):
    return ds[ds['x'] > 100*ix]\
            [ds['x'] < 100*(ix+1)]\
            [ds['y'] > 100*iy]\
            [ds['y'] < 100*(iy+1)]

In [ ]:
ds_crop = get_crop(2,1)

veg = ds_crop[ds_crop['class'] == 3]
not_veg = ds_crop[ds_crop['class'] != 3]

show_graphs([
    make_graph_3d(veg['x'], veg['y'], veg['z'], 'green'),
    make_graph_3d(not_veg['x'], not_veg['y'], not_veg['z'], 'brown')
], height=400)

## Ground mask

In [ ]:
def get_ground_mask(ds, is_imshow=False):
    points_per_km = 1000
    points_per_m = points_per_km / 1000

    not_veg = ds[ds['class'] != 3]
    x = (not_veg['x'] * points_per_m).astype(np.int)
    y = (not_veg['y'] * points_per_m).astype(np.int)
    z = not_veg['z']

    z_sums = np.histogram2d(
        y, x, 
        weights=z,
        bins=(1000, 1000), 
        range=[[0, 1000], [0, 1000]],
        normed=False,
    )[0]

    pt_counts = np.histogram2d(
        y, x, 
        bins=(1000, 1000), 
        range=[[0, 1000], [0, 1000]],
    )[0]

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    nonveg_z = np.zeros((points_per_km,points_per_km))

    m = pt_counts > 0.
    nonveg_z[m] = z_sums[m] / pt_counts[m]

    if is_imshow:
        plt.figure(figsize=(20,10))
        plt.imshow(nonveg_z)
        plt.colorbar()
        plt.show()
        
    return nonveg_z

nonveg_z = get_ground_mask(ds, is_imshow=True)

## Filter speckles in ground mask

In [ ]:
def filter_speckles_in_ground_mask(nonveg_z, is_imshow=False):
    non_zero_mask = (nonveg_z != 0.).astype(np.uint8)

    ksize = 11

    nonzero_neighbours_num = cv2.boxFilter(non_zero_mask, -1, (ksize,ksize), normalize=False)
    neighbours_sum = cv2.boxFilter(nonveg_z, -1, (ksize,ksize), normalize=False)

    nonzero_neighbours_avg = neighbours_sum / nonzero_neighbours_num

    if is_imshow:
        plt.figure(figsize=(10,10))
        plt.imshow(nonzero_neighbours_num)
        plt.colorbar()
        plt.show()
        plt.figure(figsize=(10,10))
        plt.imshow(neighbours_sum)
        plt.colorbar()
        plt.show()
        plt.figure(figsize=(10,10))
        plt.imshow(nonzero_neighbours_avg)
        plt.colorbar()
        plt.show()
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    inpaint_mask = (nonveg_z == 0.) * (nonzero_neighbours_num != 0)

    nonveg_z[inpaint_mask] = nonzero_neighbours_avg[inpaint_mask]
        
    if is_imshow:
        plt.figure(figsize=(10,10))
        plt.imshow(nonveg_z)
        plt.colorbar()
        plt.show()
        
filter_speckles_in_ground_mask(nonveg_z, is_imshow=True)

In [ ]:
def upsample_ground_mask(nonveg_z, is_imshow=False):
    points_per_km = 3000

    nonveg_z_upsampled = cv2.resize(
        nonveg_z, 
        (points_per_km, points_per_km),
        cv2.INTER_LINEAR
    )
    
    if is_imshow:
        plt.figure(figsize=(10,10))
        plt.imshow(nonveg_z_upsampled)
        plt.colorbar()
        plt.show()
        
    return nonveg_z_upsampled
        
nonveg_z_upsampled = upsample_ground_mask(nonveg_z, is_imshow=True)

## Vegetation mask

In [ ]:
def get_vegetation_mask(ds, is_imshow=False):
    points_per_km = 3000
    points_per_m = points_per_km / 1000

    veg = ds[ds['class'] == 3]
    x = np.array(
        (veg['x'] * points_per_m).astype(np.int)
    )
    y = np.array(
        (veg['y'] * points_per_m).astype(np.int)
    )
    z = np.array(veg['z'])

    veg_z_max = np.zeros((points_per_km,points_per_km))

    for i in tqdm(range(len(z))):
        xx = x[i]
        yy = y[i]
        zz = z[i]

        if zz > veg_z_max[yy,xx]:
            veg_z_max[yy,xx] = zz
            
    if is_imshow:
        plt.figure(figsize=(20,10))
        plt.imshow(veg_z_max)
        plt.colorbar()
        plt.show()
            
    return veg_z_max

veg_z_max = get_vegetation_mask(ds, is_imshow=True)

## Get vegetation heights relative to ground

In [ ]:
def get_veg_heights_25cm(nonveg_z_upsampled, veg_z_max, is_imshow=False):
    points_per_km = 3000
    
    m = (nonveg_z_upsampled != 0.) * (veg_z_max != 0.)

    veg_heights_m = np.zeros((points_per_km,points_per_km))

    veg_heights_m[m] = veg_z_max[m] - nonveg_z_upsampled[m]

    if is_imshow:
        plt.figure(figsize=(20,10))
        plt.imshow(veg_z_max, cmap='coolwarm')
        plt.colorbar()
        plt.show()
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    print('max height:', np.max(veg_heights_m))
    print('min height:', np.min(veg_heights_m))

    print('points below zero:', np.sum(veg_heights_m < 0))
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    veg_heights_m[veg_heights_m < 0] = 0

    veg_heights_25cm = (veg_heights_m * 4).astype(np.uint8)

    if is_imshow:
        plt.figure(figsize=(20,10))
        plt.imshow(veg_heights_25cm)
        plt.colorbar()
        plt.show()
        
    return veg_heights_25cm
        
veg_heights_25cm = get_veg_heights_25cm(nonveg_z_upsampled, veg_z_max, is_imshow=True)

# Process lidar dataset

In [ ]:
def process_one_lidar_file(input_path, output_path, is_imshow=False):
    
    with laspy.open(input_path) as inf:
        for points in inf.chunk_iterator(10**10):        
            x = points.x.copy()
            y = points.y.copy()
            z = points.z.copy()
            c = points.classification.copy()
            
    arr = np.stack([x,y,z,c], axis=1)
    ds = pd.DataFrame(arr, columns=['x','y','z','class'])
    
    print('Loaded dataset with %d points' % len(ds))
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    ds['x'] -= np.min(ds['x'])
    ds['y'] -= np.min(ds['y'])
    ds['z'] -= np.min(ds['z'])
    
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    print('Generating not_veg height map')
    nonveg_z = get_ground_mask(ds, is_imshow=is_imshow)
        
    # ~~~~~~~~~~~
    print('Improving not_veg height map')
    filter_speckles_in_ground_mask(nonveg_z, is_imshow=is_imshow)
    
    nonveg_z_upsampled = upsample_ground_mask(nonveg_z, is_imshow=is_imshow)
        
    # ~~~~~~~~~~~
    print('Generating veg height map')
    veg_z_max = get_vegetation_mask(ds, is_imshow=is_imshow)
    
    # ~~~~~~~~~~~
    print('Generating height map of vegetation relative to the ground')
    veg_heights_25cm = get_veg_heights_25cm(nonveg_z_upsampled, veg_z_max, is_imshow=is_imshow)
        
    cv2.imwrite(output_path, veg_heights_25cm)

In [ ]:
output_dir = f'{base_dir}/processed/lidar'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
for idx, name in enumerate(lidar_file_names):
    print(f'~~~~~~~~{idx}, Processing file {name}~~~~~~~~')
    
    input_path = os.path.join(lidar_dir, name)
    
    name_png = name[:-4] + '.png'
    output_path = os.path.join(output_dir, name_png)

    process_one_lidar_file(input_path, output_path, is_imshow=False)
    
#     break

# Process images dataset

In [ ]:
def process_one_image_file(input_path, is_imshow=False):
    img = cv2.imread(input_path)
    img = cv2.flip(img, 0)
    
    points_per_km = 3000
    
    img_downscaled = cv2.resize(
        img,
        (points_per_km, points_per_km),
        cv2.INTER_AREA
    )
    
    if is_imshow:
        plt.figure(figsize=(20,10))
        plt.imshow(img_downscaled)
        plt.show()
        
    return img_downscaled
        
img_downscaled = process_one_image_file(
    input_path=f'{image_dir}/{image_file_names[-1]}',
    is_imshow=True
)

In [ ]:
output_dir = f'{base_dir}/processed/image'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
for idx, name in enumerate(image_file_names):
    print(f'~~~~~~~~{idx}, Processing file {name}~~~~~~~~')
    
    input_path = os.path.join(image_dir, name)
    
    name_png = name.split('_')[2].replace('-', '_') + '.png'
    output_path = os.path.join(output_dir, name_png)
    
    print(output_path)

    img_downscaled = process_one_image_file(input_path, is_imshow=False)
    
    cv2.imwrite(output_path, img_downscaled)
    
#     break

# Validation

In [ ]:
output_dir = f'{base_dir}/processed/validation'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
proc_lidar_file_names = os.listdir(f'{base_dir}/processed/lidar')

for idx, name in enumerate(proc_lidar_file_names):
    print(f'~~~~~~~~{idx}, Processing file {name}~~~~~~~~')
    
    lidar_path = f'{base_dir}/processed/lidar/{name}'
    image_path = f'{base_dir}/processed/image/{name}'
    valid_path = f'{base_dir}/processed/validation/{name}'
    
    lidar = cv2.imread(lidar_path, 0)
    img = cv2.imread(image_path)
    
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    combo = np.zeros_like(img)

    combo[...,0] = img_gray
    combo[...,2] = lidar
    
    valid = np.hstack([combo, img])
    
    cv2.imwrite(valid_path, valid)